In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# GeoPandas


<a id="section_toc"></a> 
## Tabla de Contenidos

[Intro](#section_intro)

[Datos](#section_datos)

$\hspace{.5cm}$[Puntos](#section_puntos)

$\hspace{.5cm}$[Polígonos](#section_poligonos)

$\hspace{.5cm}$[Punto](#section_punto)

[Generar una linea entre dos puntos](#section_linea_punto)

[Calcular la distancia entre dos puntos](#section_distancia)

[Determinar si un punto se encuentra dentro de un polígono](#section_punto_poligono)

[Dada una línea, conocer qué poligonos atraviesa](#section_linea_poligono)

---


## Geopandas - Operaciones

<a id="section_intro"></a> 
###  Intro
[volver a TOC](#section_toc)

Como vimos anteriormente, *Geopandas* usa como estructura de datos las *GeoSeries* y *GeoDataFrame*, los que contienen al menos una columna con tipos de datos geoespaciales. Esta columna se la llama por default *geometry*.

La columna *geometry* contiene una forma geométrica que representa la posición espacial del objeto. Las formas geométricas más comunes son:
- <i>POINT</i>. Un punto
- <i>LINESTRING</i>. Una línea
- <i>POLYGON</i>. Una superficie

Es interesante realizar operaciones entre los distintos objetos usando su posición espacial.
Vamos a ver algunas de ellas:

- Generar una línea entre dos puntos.
- Determinar si un punto se encuentra dentro de un polígono.
- Dada una línea, conocer qué poligonos atraviesa.
- Calcular la distancia entre dos puntos.

<a id="section_datos"></a> 
### Datos
[volver a TOC](#section_toc)

Antes de realizar las operaciones, vamos a ver los datos de ejemplo: localizaciones de Buenos Aires representadas por puntos y polígonos.

<a id="section_puntos"></a> 
#### Puntos
[volver a TOC](#section_toc)

Los **puntos** representan algunos locales bailables de Buenos Aires.

Observar que la latitud y la longitud vienen como datos numéricos (*columnas X,Y*), y se transforman en una figura geométrica que se almacena en la columna *geometry*. 

In [ ]:
df_locales = pd.read_csv("../Data/locales-bailables.csv", encoding='latin1')
geo_locales = gpd.GeoDataFrame(df_locales, geometry = gpd.points_from_xy(df_locales.X, df_locales.Y))
geo_locales.head()

<a id="section_poligonos"></a> 
#### Polígonos

Los **polígonos** representan los barrios de Buenos Aires.

El archivo barrios.csv es del tipo *GeoCSV*, con el formato habitual para los csv pero con una columna del tipo de datos geoespaciales *WKT* con las formas geométricas, donde se indican los contornos o límites de cada barrio.

Pero la variable barrios generada a partir de la lectura del archivo es un *DataFrame, NO un GeoDataFrame*. Se debe transformar.

In [ ]:
barrios = pd.read_csv("../Data/barrios.csv", encoding='latin1')
barrios.head()

In [ ]:
type(barrios)

Para convertir de DataFrame a GeoDataFrame a *barrios*, debemos transformar la columna WKT en una columna del tipo *geometry*, el tipo de datos geoespaciales que requiere GeoDataFrame.

Para ello, necesitamos la libreria **shapely**. Es la que usa Geopandas para realizar las operaciones entre datos geoespaciales.

<div id="caja4" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/haciendo_foco.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label>GeoPandas, a través de la libreria Shapely, permite multiples operaciones geográficas. Por ejemplo: equals, contains, intersects, within, difference, union, overlaps, etc. <br> Para más información, ver <a href="https://shapely.readthedocs.io/en/stable/manual.html" target="_blank">https://shapely.readthedocs.io/en/stable/manual.html</a> </label></div>
</div>

In [ ]:
barrios.dtypes

In [ ]:
import shapely.wkt
barrios["WKT"] = barrios["WKT"].apply(shapely.wkt.loads) 
geo_barrios = gpd.GeoDataFrame(barrios, geometry='WKT')

In [ ]:
type(geo_barrios)

In [ ]:
barrios.dtypes

In [ ]:
geo_barrios.plot()

<a id="section_punto"></a> 
#### Punto

A veces necesitamos ingresar la locacion de un lugar en particular.

Por ejemplo, vamos a crear un GeoDataFrame que contenga un *Punto*, que representa el dato geoespacial de la sede de Digital House.

En este caso, vamos a crear un Punto con la asistencia de *shapely*. Y luego lo transformamos en un DataGeoFrame para poder graficarlo.

In [ ]:
from shapely.geometry import Point
p = Point( -58.443555,-34.548921) # dato geoespacial de DH
df = pd.DataFrame(data = {'id': [1]}) # Armo un DataFrame
geo_dh = gpd.GeoDataFrame(df, geometry=[p]) # lo transformo en GeoDataFrame
geo_dh

In [ ]:
type(geo_dh)

Graficamos el punto que representa a Digital House sobre los barrios de Buenos Aires

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
geo_barrios.plot(ax=ax, color='white', edgecolor='black')
geo_dh.plot(ax=ax, marker='o', color='red', markersize=40)
plt.show();

<div id="comisarias" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/ponete_a_prueba.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label>Leer el archivo comisarias.csv, y generar la columna <i>geometry</i> que representa el dato geoespacial de cada comisaria.</label></div>
</div>

---

<a id="section_linea_punto"></a> 
### Generar una línea entre dos puntos
[volver a TOC](#section_toc)

A partir de dos *puntos* que representan dos lugares, podemos trazar la *línea* que los une.

Supongamos que queremos hacer una recorrida por algunos locales bailables. Empezamos en "REY CASTRO", luego "AFRIKA", "LA CITY" y terminamos el mismo día en "PACHA BS AS". 

Queremos conocer el recorrido lineal que vamos a hacer.

Tenemos que transformar las coordenadas de las columnas X e Y  de los locales bailables, en un tipo de datos Point, para generar la línea entre los puntos.

Observar que el tipo de datos *Point* es distinto al tipo de datos *geometry*.

In [ ]:
punto_rey_castro = Point(geo_locales[geo_locales['NOMBRE']=='REY CASTRO'].X,
                         geo_locales[geo_locales['NOMBRE']=='REY CASTRO'].Y)

punto_afrika = Point(geo_locales[geo_locales['NOMBRE']=='AFRIKA'].X,
                     geo_locales[geo_locales['NOMBRE']=='AFRIKA'].Y)

punto_la_city = Point(geo_locales[geo_locales['NOMBRE']=='LA CITY'].X,
                      geo_locales[geo_locales['NOMBRE']=='LA CITY'].Y)

punto_pacha = Point(geo_locales[geo_locales['NOMBRE']=='PACHA BS AS'].X,
                    geo_locales[geo_locales['NOMBRE']=='PACHA BS AS'].Y)

In [ ]:
punto_afrika.type

Y ahora sí creamos las **líneas** entre dos puntos, usando el método **LineString** de la libreria shapely.

In [ ]:
from shapely.geometry import LineString
linea_a = LineString([punto_rey_castro, punto_afrika])
linea_b = LineString([punto_afrika, punto_la_city])
linea_c = LineString([punto_la_city, punto_pacha])

In [ ]:
linea_a.type

Finalmente, graficamos las líneas sobre los barrios de Buenos Aires.

Pero para graficar, necesitamos el método *plot* que se *aplica* a las GeoSeries y GeoDataFrame.

Nosotros tenemos hasta ahora puntos y líneas, con los tipos de datos *Point y LineString* respectivamente. Tenemos que generar una GeoSerie con los dos puntos y la línea que los vincula para poder graficarlos.

In [ ]:
linea_a_geo = gpd.GeoSeries([punto_rey_castro, punto_afrika, linea_a])
linea_b_geo = gpd.GeoSeries([punto_afrika, punto_la_city, linea_b])
linea_c_geo = gpd.GeoSeries([punto_la_city, punto_pacha, linea_c])

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
geo_barrios.plot(ax=ax, color='white', edgecolor='black')
linea_a_geo.plot(ax=ax, color='red')
linea_b_geo.plot(ax=ax, color='red')
linea_c_geo.plot(ax=ax, color='red')
plt.show();

---

<a id="section_punto_poligono"></a> 
### Determinar si un punto se encuentra dentro de un polígono
[volver a TOC](#section_toc)

Con el método `GeoDataFrame.contains` podemos ver si un polígono contiene un punto.

Como ejemplo, veamos si el local bailable *REY CASTRO* se encuentra en el barrio *MONSERRAT*.

Primero generamos dos GeoDataFrame con los datos del barrio y del local bailable.

In [ ]:
geo_monserrat = geo_barrios.loc[geo_barrios["BARRIO"]=="MONSERRAT"]
geo_rey_castro = geo_locales[geo_locales['NOMBRE']=='REY CASTRO'] 

In [ ]:
type(geo_rey_castro)

Chequeamos visualmente si se encuentra "REY CASTRO" en el barrio "MONSERRAT"

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
geo_monserrat.plot(ax=ax, color='white', edgecolor='black')
geo_rey_castro.plot(ax=ax, marker='o', color='red', markersize=40)
plt.show();

Ahora lo validamos con la función *GeoDataFrame.contains*.

Observar que para el local bailable, usamos *punto_rey_castro*, el tipo de datos Point generado anteriormente. Si usamos el GeoDataFrame *geo_rey_castro*, el método contains da error.

In [ ]:
geo_monserrat.contains(punto_rey_castro)

---

<a id="section_linea_poligono"></a> 
### Dada una línea, conocer qué poligonos atraviesa
[volver a TOC](#section_toc)

Pensando como figuras geométricas, una línea puede pasar por varios polígonos.

Como ejemplo, consideremos la linea que trazamos entre los locales bailables "AFRIKA" y "LA CITY" que generamos con la siguiente sentencia:

`linea_b = LineString([punto_afrika, punto_la_city])`

Y analicemos que barrios atraviesa.

El método **crosses** aplicado al GeoDataFrame de los barrios, indica *True* o *False* de acuerdo a si la línea atraviesa o no cada barrio (polígono). Es decir, devuelve un dato tipo *boolean* para indicar si pasa por el polígono.

Podemos observar que para los primeros barrios, solo atraviesa al barrio que tiene índice 0.

In [ ]:
geo_barrios.crosses(linea_b).head()

Ahora aplicamos el resultado anterior para obtener los barrios por donde pasa.

In [ ]:
geo_barrios.loc[geo_barrios.crosses(linea_b), 'BARRIO']

Lo podemos verificar visualmente

In [ ]:
options = ['CHACARITA','COLEGIALES','VILLA ORTUZAR','PALERMO','RECOLETA'] 
geo_barrios_recorrida = geo_barrios[geo_barrios['BARRIO'].isin(options)] 
geo_barrios_recorrida

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
geo_barrios_recorrida.plot(ax=ax, color='white', edgecolor='black')
linea_b_geo.plot(ax=ax, color='red')
plt.show();

---

<a id="section_distancia"></a> 
### Calcular la distancia entre dos puntos
[volver a TOC](#section_toc)

Ahora que sabemos como trazar líneas entre dos puntos, es razonable preguntarse cual es la distancia entre ellos. 

Vamos a calcular la *distancia* entre la ubicación de Digital House y todos los locales bailables. Y almacenamos la distancia como una nueva columna del GeoDataFrame geo_locales.

Comencemos calculando una distancia en particular. Por ejemplo, entre el edificio de Digital House y el local bailable "AFRIKA". 

Generamos un punto que identifique la posición geoespacial de cada lugar. Observar que para definir la latitud y longitud de cada lugar, la libreria **geopy** necesita que se exprese como una **tupla**.

In [ ]:
punto_dh_t = ( -58.443555,-34.548921) # dato geoespacial de DH
punto_afrika_t = ( -58.39239, -34.58843) # dato geoespacial de Afrika

In [ ]:
type(punto_dh_t)

Para calcular distancias entre dos puntos, usamos la libreria **geopy**, el método *distance*.

Observar que el resultado se puede expresar en diferentes unidades de medida: meters, kilometers, miles.

In [ ]:
import geopy.distance
dist = geopy.distance.geodesic(punto_dh_t,punto_afrika_t)
dist.meters

In [ ]:
dist.kilometers

Ahora si, calculamos todas las distancias entre Digital House y los locales bailables, y la guardamos como una nueva columna *distancia* en el GeoDataFrame geo_locales.

In [ ]:
for i in geo_locales.index:
    geo_locales.loc[i, 'distancia'] = geopy.distance.geodesic(punto_dh_t,
                  (geo_locales.loc[i,'geometry'].x
                  ,geo_locales.loc[i,'geometry'].y)).meters

In [ ]:
geo_locales.loc[:4, ['distancia','geometry','X', 'Y','NOMBRE','CALLE','NUMERO']]

Hoy a la salida de clase quiero ir a bailar al local bailable que esté más cerca (ciencia ficción!!):

In [ ]:
geo_locales.loc[geo_locales['distancia'].idxmin(),:]

---

<div id="distancia" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/ponete_a_prueba.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label>Calcular la distancia entre los locales bailables 'LA CITY' y 'PACHA BS AS'.</label></div>
</div>

---

### Referencias

Geopandas Documentacion

https://geopandas.org/gallery/create_geopandas_from_pandas.html

The GeoPandas Cookbook

https://www.martinalarcon.org/2018-12-31-d-geopandas/

The Shapely User Manual

https://shapely.readthedocs.io/en/stable/manual.html

GeoPy’s documentation

https://geopy.readthedocs.io/en/stable/